In [26]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import torch
import requests
import os
import pandas as pd
import glob

In [27]:
# url = 'images0028.png'
# image = Image.open(url)
# print(image)
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
model.classifier = torch.nn.Identity()
model.eval().to("cuda")
# inputs = processor(images=image, return_tensors="pt")
# outputs = model(**inputs)
# logits = outputs.logits

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [28]:
cols = ["name", "video", "start", "end", "speaker", "orth", "translation"]
df = pd.read_csv(
    'orgin_data/PHOENIX-2014-T-release-v3/PHOENIX-2014-T/annotations/manual/PHOENIX-2014-T.test.corpus.csv',
    sep="|",
    header=None,
    names=cols,
    dtype=str)

In [29]:
train_data = df.to_dict(orient="records")
for rec in train_data:
    rec.pop('start', None)
    rec.pop('end', None)
    rec.pop('video')
    rec['gloss'] = rec.pop('orth')

In [30]:
# FRAMES_ROOT = 'orgin_data/PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/train'
# BATCH_SIZE = 32
# for rec in train_data:
#     pattern = os.path.join(FRAMES_ROOT,rec["name"],"*.png")
#     img_paths = sorted(glob.glob(pattern))
#     frame_feats = []
#     for i in range(0, len(img_paths)):
#         image = Image.open(img_paths[i])
#         inputs = processor(images=image, return_tensors="pt")
#         outputs = model(**inputs)
#         frame_feats.append(outputs.logits)
#     # print(frame_feats)
#     # print(frame_feats.shape)
#     rec['sign'] = frame_feats
#     

In [31]:
FRAMES_ROOT = 'orgin_data/PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/test'
BATCH_SIZE = 192
for rec in train_data:
    pattern = os.path.join(FRAMES_ROOT,rec["name"],"*.png")
    img_paths = sorted(glob.glob(pattern))
    frame_feats = []
    for i in range(0, len(img_paths), BATCH_SIZE):
        # ⚠ 建议批处理而不是逐帧（这里演示批量）
        batch_imgs = [
            Image.open(p).convert("RGB") for p in img_paths[i : i + BATCH_SIZE]
        ]
        inputs = processor(images=batch_imgs, return_tensors="pt")
        # 若模型在 GPU，别忘了把 inputs 迁移同设备
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():             # 推理阶段关闭梯度
            out = model(**inputs).logits.detach().cpu()  # (B, 768)
        frame_feats.append(out)
    # print(frame_feats)
    # print(frame_feats.shape)
    rec['sign'] = frame_feats
    print(rec["name"]+"已经处理完成")
                 
    

name已经处理完成
25October_2010_Monday_tagesschau-17已经处理完成
25October_2010_Monday_tagesschau-24已经处理完成
15December_2010_Wednesday_tagesschau-37已经处理完成
10March_2011_Thursday_heute-58已经处理完成
14August_2009_Friday_tagesschau-62已经处理完成
14August_2009_Friday_tagesschau-73已经处理完成
14August_2009_Friday_tagesschau-75已经处理完成
26January_2010_Tuesday_heute-105已经处理完成
26January_2010_Tuesday_heute-106已经处理完成
01November_2010_Monday_tagesschau-133已经处理完成
01November_2010_Monday_tagesschau-136已经处理完成
01November_2010_Monday_tagesschau-137已经处理完成
08May_2010_Saturday_tagesschau-175已经处理完成
08May_2010_Saturday_tagesschau-176已经处理完成
24November_2011_Thursday_heute-224已经处理完成
21August_2011_Sunday_tagesschau-263已经处理完成
22November_2010_Monday_heute-283已经处理完成
14October_2010_Thursday_tagesschau-287已经处理完成
14October_2010_Thursday_tagesschau-290已经处理完成
14October_2010_Thursday_tagesschau-291已经处理完成
14October_2010_Thursday_tagesschau-294已经处理完成
06April_2010_Tuesday_tagesschau-306已经处理完成
12May_2010_Wednesday_tagesschau-323已经处理完成
12April_2010_Monday_t

In [32]:
import gzip,pickle
for rec in train_data:
    if "frame_feats" in rec and isinstance(rec["frame_feats"], torch.Tensor):
        rec["frame_feats"] = rec["frame_feats"].cpu()
outfile = "test_data.pkl.gz"
with gzip.open(outfile, "wb") as f:
    # 用最高协议序列化并压缩
    pickle.dump(train_data, f, protocol=pickle.HIGHEST_PROTOCOL)
print(f"✅ 数据已保存为 {outfile}")

✅ 数据已保存为 test_data.pkl.gz


In [33]:
train_data

[{'name': 'name',
  'speaker': 'speaker',
  'translation': 'translation',
  'gloss': 'orth',
  'sign': []},
 {'name': '25October_2010_Monday_tagesschau-17',
  'speaker': 'Signer01',
  'translation': 'regen und schnee lassen an den alpen in der nacht nach im norden und nordosten fallen hier und da schauer sonst ist das klar',
  'gloss': 'REGEN SCHNEE REGION VERSCHWINDEN NORD REGEN KOENNEN REGION STERN KOENNEN SEHEN',
  'sign': [tensor([[ 0.0568,  0.6757,  0.5213,  ..., -0.6768, -0.6291, -0.6087],
           [-0.0209,  0.5255,  0.2856,  ..., -0.9771, -0.9689, -0.6742],
           [-0.5286,  0.2422,  0.0111,  ..., -1.1138, -0.6597, -0.1992],
           ...,
           [-0.2739,  0.2915, -0.0305,  ..., -1.2537, -1.0979, -0.6763],
           [ 0.0715, -0.0705,  0.1062,  ..., -0.8247, -1.2514, -0.7248],
           [ 0.2865,  0.1986,  0.1876,  ..., -1.0508, -1.2633, -0.7873]])]},
 {'name': '25October_2010_Monday_tagesschau-24',
  'speaker': 'Signer01',
  'translation': 'am donnerstag regen in